In [1]:
import ast
import os
import sys
import tqdm
import pandas as pd

from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from pathlib import Path

sys.path.insert(0, str(Path().resolve().parent))

from app.services.Milvus.milvus_retriver import HybridRetriever
from app.services.rag_pipeline.rag_pipeline import AnswerGenerator

In [3]:
testdataset = pd.read_csv('~/Work/data/q-insigt-testset.csv', index_col=False)

In [4]:
testdataset.head()

,user_input,reference_contexts,reference,synthesizer_name
0,Что произошло с Габеном в последнее время?,"['POV – казуальный Габен, когда все ждут геймп...","Недавно Габен стал казуальным, когда все ждали...",single_hop_specifc_query_synthesizer
1,Что такое Kafka и почему это важно для работы ...,['#вакансия #Middle #DataAnalyst #fulltime #уд...,"Kafka — это система, которая используется для ...",single_hop_specifc_query_synthesizer
2,Что такое Висп и почему его упоминают вместе с...,['Снова что-то на квинновском 🆗 Легко говорить...,Висп упоминается в контексте Лешего как одна и...,single_hop_specifc_query_synthesizer
3,Какой контракт выиграла Lockheed Martin и на к...,['Lockheed Martin выиграла контракт на противо...,Lockheed Martin выиграла контракт на противора...,single_hop_specifc_query_synthesizer
4,Кто получает мерч?,"['Привет, Чемпионы!🏆\n\nСегодня произошло знам...","Мерч получает каждый, кто получил сертификат и...",single_hop_specifc_query_synthesizer


In [5]:
testdataset['text'] = testdataset.reference_contexts.apply(lambda x: ast.literal_eval(x)[0])

In [6]:
data_path = "/Users/danilovsnnv/Work/data/telegram_posts.csv"
df = pd.read_csv(data_path)
df = df.dropna(subset=['text'])

In [7]:
df.head()

,date,name,text,views,comments,forwards,emoji,reactions,subscribers
0,2024-06-27 14:59:34,🏄 Соревновательный Data Science | Kaggle | Чем...,"**👨‍💻**** Апдейт про Kaggle Camp, который вы ж...",755.0,2,3,"👍: 14, ⚡: 7, ❤‍🔥: 3",24,2672
1,2024-06-25 15:33:36,🏄 Соревновательный Data Science | Kaggle | Чем...,👨‍💻 **Контент из сообщества**! \n**\n****🏄****...,877.0,3,19,"👍: 16, Custom emoji: 7",23,2672
2,2024-06-24 12:06:49,🏄 Соревновательный Data Science | Kaggle | Чем...,🏆 [**Запись стрима**](https://youtu.be/kz__54Y...,1075.0,0,11,"❤‍🔥: 10, Custom emoji: 2",12,2672
3,2024-06-21 14:10:38,🏄 Соревновательный Data Science | Kaggle | Чем...,Стрим случился! Всех причастных рады были увид...,1348.0,9,2,❤‍🔥: 4,4,2672
4,2024-06-21 12:45:52,🏄 Соревновательный Data Science | Kaggle | Чем...,"Привет, работяги! 👨‍💻\nКак неделя прошла? Есть...",1348.0,4,2,❤‍🔥: 10,10,2672


In [8]:
testdataset = testdataset.merge(df[['text', 'name']], on='text')

In [9]:
testdataset.head()

,user_input,reference_contexts,reference,synthesizer_name,text,name
0,Что произошло с Габеном в последнее время?,"['POV – казуальный Габен, когда все ждут геймп...","Недавно Габен стал казуальным, когда все ждали...",single_hop_specifc_query_synthesizer,"POV – казуальный Габен, когда все ждут геймпле...",BetBoom Esports Dota 2
1,Что такое Kafka и почему это важно для работы ...,['#вакансия #Middle #DataAnalyst #fulltime #уд...,"Kafka — это система, которая используется для ...",single_hop_specifc_query_synthesizer,#вакансия #Middle #DataAnalyst #fulltime #удал...,Data Science Jobs
2,Что такое Висп и почему его упоминают вместе с...,['Снова что-то на квинновском 🆗 Легко говорить...,Висп упоминается в контексте Лешего как одна и...,single_hop_specifc_query_synthesizer,"Снова что-то на квинновском 🆗 Легко говорить, ...",BetBoom Esports Dota 2
3,Какой контракт выиграла Lockheed Martin и на к...,['Lockheed Martin выиграла контракт на противо...,Lockheed Martin выиграла контракт на противора...,single_hop_specifc_query_synthesizer,Lockheed Martin выиграла контракт на противора...,"Больше, чем экономика"
4,Кто получает мерч?,"['Привет, Чемпионы!🏆\n\nСегодня произошло знам...","Мерч получает каждый, кто получил сертификат и...",single_hop_specifc_query_synthesizer,"Привет, Чемпионы!🏆\n\nСегодня произошло знамен...",🏄 Соревновательный Data Science | Kaggle | Чем...


In [10]:
import os
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent))

from app.services.Milvus.milvus_retriver import HybridRetriever
from app.services.rag_pipeline.rag_pipeline import AnswerGenerator

In [11]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
retriever = HybridRetriever(
    uri="http://localhost:19530",
    collection_name="main",
    dense_embedding_function=embeddings
)
retriever.build_collection(recreation=False)

2025-01-25 14:20:16,512 [ERROR][_create_connection]: Failed to create new connection using: 4bc36ba0e11341c8ac560bb4b7009c72 (milvus_client.py:848)


MilvusException: <MilvusException: (code=2, message=Fail connecting to server on localhost:19530, illegal connection params or server unavailable)>

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
retriever = HybridRetriever(
    uri="http://localhost:19530",
    collection_name="main",
    dense_embedding_function=embeddings
)
retriever.build_collection(recreation=True)

for index, row in tqdm.tqdm(df.head().iterrows(), total=5): # df.shape[0]):
    retriever.insert_data(
        {"message": row['text'], 'chat_name': row['name'], 'chat_message_id': index}
    )

In [16]:
generator = AnswerGenerator(retriever)
retriever_params = {
    'k': 1,
    # 'mode': 'hybrid',
    # 'weights': [0.5, 0.5],
}

answers = []
testdataset_sample = testdataset.head()
for index, row in tqdm.tqdm(testdataset_sample.iterrows(), total=testdataset_sample.shape[0]):
    answer = await generator.generate_answer(
        row['user_input'],
        chat_names=row['name'],
        retriever_params=retriever_params
    )
    answers.append(answer)


 20%|█████████                                    | 1/5 [00:05<00:23,  5.83s/it]

BetBoom Esports Dota 2


 40%|██████████████████                           | 2/5 [00:07<00:10,  3.61s/it]

Data Science Jobs


 60%|███████████████████████████                  | 3/5 [00:10<00:06,  3.30s/it]

BetBoom Esports Dota 2


 80%|████████████████████████████████████         | 4/5 [00:12<00:02,  2.61s/it]

Больше, чем экономика


100%|█████████████████████████████████████████████| 5/5 [00:13<00:00,  2.66s/it]

🏄 Соревновательный Data Science | Kaggle | Чемпионаты


In [13]:
retriever_params = {
    'k': 1,
    # 'mode': 'hybrid',
    # 'weights': [0.5, 0.5],
}

retrieved_documents = await retriever.search(
    query='Что такое Kafka?',
    chat_names='Соревновательный Data Science | Kaggle | Чемпионаты',
    **retriever_params
)

In [14]:
retrieved_documents

[]

In [17]:
answers

['Извините, но у меня нет информации о событиях, связанных с Габеном, так как контекст не был предоставлен.',
 'Kafka — это распределённая платформа потоковой передачи данных, которая позволяет обрабатывать и передавать данные в реальном времени. Она важна для работы аналитика данных, так как обеспечивает высокую пропускную способность, надёжность и возможность обработки больших объёмов данных с минимальной задержкой. Это позволяет аналитикам получать актуальную информацию и принимать решения на основе свежих данных, что критично в условиях быстро меняющегося бизнес-окружения.',
 "Висп — это персонаж из игры Dota 2, который часто упоминается в контексте имб (имбалансных) героев. Его способности позволяют ему эффективно контролировать игру и оказывать значительное влияние на исход сражений. Леший, также известный как Nature's Prophet, является другим героем, который может быть упомянут в этом контексте из-за своей способности к быстрой мобильности и нанесению урона. Упоминание Виспа вме